In [ ]:
from itertools import combinations
import numpy as np
import sys
from oxDNA_analysis_tools.UTILS.oxview import oxdna_conf, from_path
from oxDNA_analysis_tools.UTILS.RyeReader import describe, get_confs, inbox
from oxDNA_analysis_tools.UTILS.data_structures import TopInfo, TrajInfo
from pathlib import Path
import os
from ipy_oxdna.dna_structure import DNAStructure, DNAStructureStrand, load_dna_structure, DNABase, strand_from_info
from copy import deepcopy
from ipy_oxdna.oxdna_simulation import Simulation, SimulationManager
import copy
from tqdm.auto import tqdm
from ipy_oxdna.oxdna_simulation import Simulation, SimulationManager
import os
import random

In [ ]:
input_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/unmodified/1512_bp'

def load_dna_structure_files(input_path):
    dat_path = os.path.join(input_path, '1512_bp.dat')
    top_path = os.path.join(input_path, '1512_bp.top')
    dna = load_dna_structure(top_path, dat_path)
    return dna

In [ ]:
dna = load_dna_structure_files(input_path)

In [ ]:
def find_longest_strand(dna):
    longest_strand = None
    longest_strand_index = -1
    max_length = 0
    for index, strand in enumerate(dna.strands):
        if len(strand.bases) > max_length:
            max_length = len(strand.bases)
            longest_strand = strand
            longest_strand_index = index
    return longest_strand, longest_strand_index

In [ ]:
longest_strand, longest_strand_index = list(find_longest_strand(dna))      # Has been checked with the oxview information and they match
print ("longest_strand:",longest_strand)
print("longest_strand_index:",longest_strand_index)
print("Number of bases in longest strand:",len(longest_strand))

In [ ]:
def find_cross_over_in_longest_strand(longest_strand):
    min_distance = float('inf')
    max_index_difference = 0
    cross_over_bases_max = (None, None)
    num_bases = len(longest_strand)
    for i in range(num_bases):
        for j in range(i + 1, num_bases):
            base_i = longest_strand[i]
            base_j = longest_strand[j]
            index_difference = abs(base_i.uid - base_j.uid)
            distance = np.linalg.norm(np.array(base_i.pos) - np.array(base_j.pos))
            if index_difference > max_index_difference or (index_difference == max_index_difference and distance < min_distance):
                max_index_difference = index_difference
                min_distance = distance
                cross_over_bases_max = (base_i, base_j)
    return cross_over_bases_max, max_index_difference, min_distance

In [ ]:
cross_over_bases_max, max_index_difference, min_distance = find_cross_over_in_longest_strand(longest_strand)          # Has been checked with the oxview info and they match
print("cross_over_bases_max:",cross_over_bases_max)
print("max_index_difference:", max_index_difference)
print("min_distance:", min_distance)

In [ ]:
left_indices = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1411,1412,1413,1414,1415,1416,1417,1418,1419,1420,1421,1422,1423,1424,1425,1426,1427,1428,1429,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439,1440,1441,1442,1443,1444,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454,2293,2294,2295,2301,2302,2303,2304,2305,2306,2307,2308,2309,2310,2311,2312,2313,2314,2315,2316,2317,2318,2319,2320,2321,2322,2323,2324,2325,2326,2327,2328,2329,2330,2331,2332,2333,2334,2335,2336,2337,2338,2339,2340,2341,2342,2343,2344,2345,2346,2347,2348,2349,2350,2351,2352,2353,2354,2355,2356,2357,2358,2359,2360,2361,2362,2363,2364,2365,2366,2367,2368,2369,2370,2371,2372,2373,2374,2375,2376,2377,2378,2379,2380,2381,2783,2784,2785,2786,2787,2788,2789,2802,2803,2804,2805,2806,3011,3012,3013,3014,3015,3016,3017,3018,3019,3020,3021,3022,3023,3024,3025,3026,3027,3028,3029,3030,3031,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067]
right_indices = [20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1155,1156,1157,1158,1159,1160,1161,1162,1163,1164,1165,1166,1167,1168,1169,1170,1171,1172,1173,1174,1175,1176,1177,1178,1179,1180,1181,1182,1183,1184,1185,1186,1187,1188,1189,1190,1191,1192,1193,1194,1195,1196,1197,1198,1199,1555,1556,1557,1558,1559,1560,1561,1562,1563,1564,1565,1566,1567,1568,1569,1570,1571,1572,1573,1574,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1603,1604,1605,1606,1607,1608,1609,1610,1611,1612,1613,1614,1615,1616,1617,1618,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1630,1631,1632,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1667,1668,1669,1670,2382,2383,2384,2385,2386,2387,2388,2389,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399,2400,2401,2402,2403,2404,2405,2406,2407,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417,2418,2419,2420,2421,2422,2423,2424,2425,2426,2427,2428,2429,2430,2431,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441,2442,2443,2444,2445,2446,2447,2448,2449,2450,2451,2452,2453,2454,2455,2456,2457,2458,2459,2460,2461,2462,2463,2464,2465,2466,2467,2479,2480,2481,2482,2483,2484,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500,2501,2502,2818,2819,2820,2821,2822,2823]

def calculate_left_right_pos(dna, left_indices, right_indices):
    left_pos = []
    right_pos = []
    
    for strand in dna.strands:
        for base in strand:
            if base.uid in left_indices:
                left_pos.append(base.pos)
            elif base.uid in right_indices:
                right_pos.append(base.pos)
    
    if left_pos:
        cms_left_side = np.mean(left_pos, axis=0)
    else:
        raise ValueError("No positions found for left indices.")
    
    if right_pos:
        cms_right_side = np.mean(right_pos, axis=0)
    else:
        raise ValueError("No positions found for right indices.")
    
    print(f"Center of mass for the left side: {cms_left_side}")
    print(f"Center of mass for the right side: {cms_right_side}")
    
    midpoint = (cms_left_side + cms_right_side) / 2
    print(f"Midpoint between the left and right sides: {midpoint}")
    
    return cms_left_side, cms_right_side, midpoint

In [ ]:
cms_left, cms_right, midpoint = calculate_left_right_pos(dna, left_indices, right_indices)

In [ ]:
def is_point_far_from_crossovers(point, crossover_positions, min_distance_threshold):
    for pos in crossover_positions:
        distance = np.linalg.norm(np.array(point) - np.array(pos))
        if distance < min_distance_threshold:
            return False
    return True

In [ ]:

def find_valid_point(dna, left_indices, right_indices, longest_strand, min_distance_threshold=4):
    cms_left_side, cms_right_side, midpoint = calculate_left_right_pos(dna, left_indices, right_indices)
    
    cross_over_bases, max_index_difference, min_distance = find_cross_over_in_longest_strand(longest_strand)
    crossover_positions = [base.pos for base in cross_over_bases if base is not None]
    
    t = random.uniform(0, 1)
    first_P = np.array(cms_left_side + t * (cms_right_side - cms_left_side))
    if not crossover_positions:
        return first_P
    
    while True:
        t = random.uniform(0, 1)
        P = np.array(cms_left_side + t * (cms_right_side - cms_left_side))
        if is_point_far_from_crossovers(P, crossover_positions, min_distance_threshold):
            return P

In [ ]:
# Find some point on both left and right sides of the point P to calculate the bent angle 
def find_bases_around_point(dna, point, min_distance, max_distance):
    left_bases = []
    right_bases = []
    for strand in dna.strands:
        for base in strand:
            distance = np.linalg.norm(np.array(base.pos) - np.array(point))
            if min_distance < distance < max_distance:
                if base.pos[0] < point[0]:
                    left_bases.append(base.pos)
                else:
                    right_bases.append(base.pos)
                    
    # print(f"Left bases: {left_bases}")
    # print(f"Right bases: {right_bases}")
    
    if left_bases:
        cms_left_bases = np.mean(left_bases, axis=0)
        print(f"Center of mass for left bases: {cms_left_bases}")
    else:
        cms_left_bases = None
        print("No left bases found.")
    
    if right_bases:
        cms_right_bases = np.mean(right_bases, axis=0)
        print(f"Center of mass for right bases: {cms_right_bases}")
    else:
        cms_right_bases = None
        print("No right bases found.")
    
    return left_bases, right_bases, cms_left_bases, cms_right_bases

In [ ]:
min_distance = 2.0  
max_distance = 7.0
P = find_valid_point(dna, left_indices, right_indices, longest_strand)
print(f"Valid point P: {P}")
left_bases, right_bases, cms_left_bases, cms_right_bases = find_bases_around_point(dna, P, min_distance, max_distance)

In [ ]:
def calculate_center_of_mass(positions):
    if not positions:
        raise ValueError("No positions provided for center of mass calculation.")
    return np.mean(positions, axis=0)

In [ ]:
def calculate_bend_angle(P, cms_left, cms_right):
    vec_left = cms_left - P
    vec_right = cms_right - P
    unit_vec_left = vec_left / np.linalg.norm(vec_left)
    unit_vec_right = vec_right / np.linalg.norm(vec_right)
    dot_product = np.dot(unit_vec_left, unit_vec_right)
    angle = np.arccos(dot_product) * (180.0 / np.pi)
    return angle

In [ ]:
def find_bend_angle(dna, left_indices, right_indices, longest_strand, min_distance_threshold=4, min_distance=2.0, max_distance=7.0):
    point_pos = find_valid_point(dna, left_indices, right_indices, longest_strand, min_distance_threshold)
    left_bases, right_bases, cms_left_bases, cms_right_bases = find_bases_around_point(dna, P, min_distance, max_distance)  
    cms_left = calculate_center_of_mass(left_bases)
    cms_right = calculate_center_of_mass(right_bases)
    bend_angle = calculate_bend_angle(point_pos, cms_left, cms_right)
    return point_pos, bend_angle

In [ ]:
P_pos, bend_angle = find_bend_angle(dna, left_indices, right_indices, longest_strand, min_distance_threshold=5.0, min_distance=5.0, max_distance=10.0)
print(f"Point P position: {P_pos}")
print(f"Bend angle at point P: {bend_angle} degrees")

In [ ]:
def calculate_angles_for_all_structures(dna_list, left_indices, right_indices, min_distance_threshold=2.0, min_distance=5.0, max_distance=10.0):
    angles = []
    for dna in dna_list:
        longest_strand, _ = find_longest_strand(dna)
        point_pos, bend_angle = find_bend_angle(dna, left_indices, right_indices, longest_strand, min_distance_threshold, min_distance, max_distance)
        angles.append((point_pos, bend_angle))
    return angles

In [ ]:
sphere_radius = 3.5   # I checked the radius of the bundle from the oxview and chose this number for the sphere radius

def find_bases_in_sphere(dna, point, sphere_radius):
    bases_in_sphere = []
    base_to_strand_mapping = {}
    for strand_index, strand in enumerate(dna.strands):
        for base in strand:
            base_position = np.array(base.pos)
            distance = np.linalg.norm(base_position - point)
            if distance < sphere_radius:
                bases_in_sphere.append(base.uid)
                base_to_strand_mapping[base.uid] = strand_index
    return bases_in_sphere, base_to_strand_mapping

In [ ]:
bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(dna, P, sphere_radius)
print('bases_in_sphere:', bases_in_sphere)
print('base_to_strand_mapping:', base_to_strand_mapping)
print('Number_bases_in_sphere:', len(bases_in_sphere))

In [ ]:
def export_dna_structures(new_dna_structures, base_path):
    output_paths = []
    for i, new_dna in enumerate(new_dna_structures):
        structure_id = i
        unique_subdir = os.path.join(base_path, f'structure_{structure_id}')
        os.makedirs(unique_subdir, exist_ok=True)
        dat_path = os.path.join(unique_subdir, '1512_bp_rmv_staples.dat')
        top_path = os.path.join(unique_subdir, '1512_bp_rmv_staples.top')
        new_dna.export_top_conf(Path(top_path), Path(dat_path))
        output_paths.append({
            'structure_id': structure_id,
            'dat_path': dat_path,
            'top_path': top_path
        })
    return output_paths

In [ ]:
def remove_one_strand_in_sphere(dna, point, sphere_radius):
    bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(dna, point, sphere_radius)
    longest_strand, longest_strand_index = find_longest_strand(dna)
    strands_to_remove = set(base_to_strand_mapping.values()) - {longest_strand_index}
    dna_structures = []
    for strand_index in strands_to_remove:
        strand_list = []
        for idx, strand in enumerate(dna.strands):
            if idx != strand_index:
                strand_list.append(strand)
        new_dna_structure = DNAStructure(strand_list, dna.time, dna.box, dna.energy)
        dna_structures.append(new_dna_structure)
    return dna_structures

In [ ]:
new_dna_structures = remove_one_strand_in_sphere(dna, P, sphere_radius)
print("new_dna_structures:",new_dna_structures)
print(len(new_dna_structures))

In [ ]:
angles = calculate_angles_for_all_structures(new_dna_structures, left_indices, right_indices)

for i, (point_pos, bend_angle) in enumerate(angles):
    print(f"Structure {i}: Point P position: {point_pos}, Bend angle: {bend_angle} degrees")

In [ ]:
base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/one_staple_remvd'
output_paths_one = export_dna_structures(new_dna_structures, base_path)

In [ ]:
def remove_two_strands_in_sphere(dna, point, sphere_radius):
    bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(dna, P, sphere_radius)
    longest_strand, longest_strand_index = find_longest_strand(dna)
    
    print("Bases in sphere:", bases_in_sphere)
    print("Base to strand mapping:", base_to_strand_mapping)
    print("Longest strand:", longest_strand)
    print("Longest strand index:", longest_strand_index)
    
    strands_to_remove = set(base_to_strand_mapping.values()) - {longest_strand_index}
    print("Strands to remove:", strands_to_remove)

    dna_structures = []
    removed_strands_info = []

    # Create all possible pairs of strands to remove
    strand_pairs = [(strand_1, strand_2) for i, strand_1 in enumerate(strands_to_remove)
                    for strand_2 in list(strands_to_remove)[i + 1:]]

    for strand_1, strand_2 in strand_pairs:
        strand_list = []
        for idx, strand in enumerate(dna.strands):
            if idx not in {strand_1, strand_2}:
                strand_list.append(strand)
        
        new_dna_structure = DNAStructure(strand_list, dna.time, dna.box, dna.energy)
        dna_structures.append(new_dna_structure)
        
        removed_strands_info.append((strand_1, strand_2))
        print(f"Removed strands: {strand_1}, {strand_2}")
    
    return dna_structures, removed_strands_info



In [ ]:
new_dna_structures_two, removed_strands_info_two = remove_two_strands_in_sphere(dna, P, sphere_radius)

print(f"Generated {len(new_dna_structures_two)} new DNA structures with two strands removed.")
# print("Details of removed strands for each structure:")
# for strands in removed_strands_info_two:
#     print(f"Strands removed: {strands}")

In [ ]:
new_dna_structures_two, removed_strands_info_two = remove_two_strands_in_sphere(dna, P, sphere_radius)
print(new_dna_structures_two)
print(len(new_dna_structures_two))

In [ ]:
base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/two_staples_remvd'
output_paths_two = export_dna_structures(new_dna_structures_two, base_path)

In [ ]:
def remove_three_strands_in_sphere(dna, point, sphere_radius):
    bases_in_sphere, base_to_strand_mapping = find_bases_in_sphere(dna, P, sphere_radius)
    longest_strand, longest_strand_index = find_longest_strand(dna)
    
    print("Bases in sphere:", bases_in_sphere)
    print("Base to strand mapping:", base_to_strand_mapping)
    print("Longest strand:", longest_strand)
    print("Longest strand index:", longest_strand_index)
    
    strands_to_remove = set(base_to_strand_mapping.values()) - {longest_strand_index}
    print("Strands to remove:", strands_to_remove)

    dna_structures = []
    removed_strands_info = []

    # Create all possible triplets of strands to remove
    strand_triplets = [(strand_1, strand_2, strand_3) for i, strand_1 in enumerate(strands_to_remove)
                       for j, strand_2 in enumerate(list(strands_to_remove)[i + 1:])
                       for strand_3 in list(strands_to_remove)[i + j + 2:]]

    for strand_1, strand_2, strand_3 in strand_triplets:
        strand_list = []
        for idx, strand in enumerate(dna.strands):
            if idx not in {strand_1, strand_2, strand_3}:
                strand_list.append(strand)
        
        new_dna_structure = DNAStructure(strand_list, dna.time, dna.box, dna.energy)
        dna_structures.append(new_dna_structure)
        
        removed_strands_info.append((strand_1, strand_2, strand_3))
        print(f"Removed strands: {strand_1}, {strand_2}, {strand_3}")
    
    return dna_structures, removed_strands_info


In [ ]:
new_dna_structures_three, removed_strands_info = remove_three_strands_in_sphere(dna, P, sphere_radius)
print(f"Generated {len(new_dna_structures_three)} new DNA structures with three strands removed.")
# print("Details of removed strands for each structure:")
# for strands in removed_strands_info:
#     print(f"Strands removed: {strands}")

In [ ]:
new_dna_structures_three, removed_strands_info = remove_three_strands_in_sphere(dna, P, sphere_radius)
print(new_dna_structures_three)
print(len(new_dna_structures_three))

In [ ]:
base_path = "/home/ava/MetaBackbone_project/Metabackbone-scripts/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/three_staples_remvd"
output_paths_three= export_dna_structures(new_dna_structures_three, base_path)

In [ ]:
file_dir = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/one_staple_remvd/structure_1'
structure_id = os.path.basename(file_dir)
print(structure_id)


In [ ]:
# Define the base simulation path
eq_steps = 1e5
prod_steps = 1e5
rel_steps = 1e3
    
# Parameters for simulations
eq_parameters = {'dt':f'0.003','steps':f'{eq_steps}','print_energy_every': f'1e5', 'interaction_type': 'DNA2',
                 'print_conf_interval':f'1e5', 'fix_diffusion':'false', 'T':f'20C','max_density_multiplier':f'50'}

prod_parameters = {'dt':f'0.003','steps':f'{prod_steps}','print_energy_every': f'1e5', 'interaction_type': 'DNA2',
                   'print_conf_interval':f'1e5', 'fix_diffusion':'false', 'T':f'20C','max_density_multiplier':f'50'}
rel_parameters = {'steps': f'{rel_steps}', 'max_backbone_force': '200', 'max_backbone_force_far': '200'}

sim_base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/metabackbone/notebooks/Simulations/simulations/Automatically_rmvd_staples/one_staple_removed'
base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/one_staple_remvd'


# Function to queue relaxation simulations
def queue_relaxation_simulations(structures, base_path, sim_base_path):
    simulation_manager = SimulationManager()
    sim_list_rel = []
    for structure_id, structure in enumerate(structures):
        file_dir = os.path.join(base_path, f'structure_{structure_id}')
        sim_path = os.path.join(sim_base_path, f'1512_bp_{structure_id}')
        rel_dir = os.path.join(sim_path, 'relaxed')
        if not os.path.exists(file_dir):
            print(f"Directory does not exist: {file_dir}")
            continue
        sim_relax = Simulation(file_dir, rel_dir)
        sim_relax.build(clean_build='force')
        sim_relax.input.swap_default_input("cpu_MC_relax")
        sim_relax.input_file(rel_parameters)
        simulation_manager.queue_sim(sim_relax)
        sim_list_rel.append(sim_relax)
        print(f"Queued relaxation simulation for structure {structure_id}")
    simulation_manager.worker_manager(gpu_mem_block=False)
    print("Completed all relaxation simulations")
    return sim_list_rel

In [ ]:
# Function to queue equilibration simulations
def queue_equilibration_simulations(structures, base_path, sim_base_path):
    simulation_manager = SimulationManager()
    sim_list_eq = []
    for structure_id, structure in enumerate(structures):
        sim_path = os.path.join(sim_base_path, f'1512_bp_{structure_id}')
        rel_dir = os.path.join(sim_path, 'relaxed')
        eq_dir = os.path.join(sim_path, 'eq')
        if not os.path.exists(rel_dir):
            print(f"Directory does not exist: {rel_dir}")
            continue
        sim_eq = Simulation(rel_dir, eq_dir)
        sim_eq.build(clean_build='force')
        sim_eq.input_file(eq_parameters)
        simulation_manager.queue_sim(sim_eq)
        sim_list_eq.append(sim_eq)
        print(f"Queued equilibration simulation for structure {structure_id}")
    simulation_manager.worker_manager(gpu_mem_block=False)
    print("Completed all equilibration simulations")
    return sim_list_eq

In [ ]:
# Function to queue production simulations
def queue_production_simulations(structures, base_path, sim_base_path):
    simulation_manager = SimulationManager()
    sim_list_prod = []
    for structure_id, structure in enumerate(structures):
        sim_path = os.path.join(sim_base_path, f'1512_bp_{structure_id}')
        eq_dir = os.path.join(sim_path, 'eq')
        prod_dir = os.path.join(sim_path, 'prod')
        if not os.path.exists(eq_dir):
            print(f"Directory does not exist: {eq_dir}")
            continue
        sim_prod = Simulation(eq_dir, prod_dir)
        sim_prod.build(clean_build='force')
        sim_prod.input_file(prod_parameters)
        simulation_manager.queue_sim(sim_prod)
        sim_list_prod.append(sim_prod)
        print(f"Queued production simulation for structure {structure_id}")
    simulation_manager.worker_manager(gpu_mem_block=False)
    print("Completed all production simulations")
    return sim_list_prod

In [ ]:
# Function to run all simulations
def run_all_simulations(structures, base_path, sim_base_path):
    print("Starting relaxation simulations...")
    sim_list_rel = queue_relaxation_simulations(structures, base_path, sim_base_path)
    print("Starting equilibration simulations...")
    sim_list_eq = queue_equilibration_simulations(structures, base_path, sim_base_path)
    print("Starting production simulations...")
    sim_list_prod = queue_production_simulations(structures, base_path, sim_base_path)
    return sim_list_rel, sim_list_eq, sim_list_prod

In [ ]:
def main(base_path, sim_base_path, target_angle=90, tolerance=5):
    initial_dna = load_dna_structure_files(base_path)
    results = []
    sphere_radius = 3.5  
    excluded_strands = set()

    longest_strand, _ = find_longest_strand(initial_dna)

    while True:
        point = find_valid_point(initial_dna, left_indices, right_indices, longest_strand)
        new_structure, removed_strand = remove_one_strand_in_sphere(initial_dna, point, sphere_radius, excluded_strands)
        # If no new structure is generated, exit the loop
        if new_structure is None:
            break
        
        # Add the removed strand to the set of excluded strands
        excluded_strands.add(removed_strand)
        
        # Export the new structure and run simulations
        output_paths = export_dna_structures([new_structure], base_path)
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations([new_structure], base_path, sim_base_path)
        
        # Find the bend angle of the new structure
        point_pos, bend_angle = find_bend_angle(new_structure, left_indices, right_indices, longest_strand)
        results.append((point_pos, bend_angle))
        
        # Check if the bend angle is within the desired tolerance
        if abs(bend_angle - target_angle) <= tolerance:
            print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
            return results
    
    return results



In [ ]:
def main(base_path, sim_base_path, left_indices, right_indices, target_angle=90, tolerance=5):
    initial_dna = load_dna_structure_files(base_path)
    modified_structures = [initial_dna]
    results = []
    sphere_radius = 3.5  # Define your sphere radius
    while True:
        new_structures = []
        for dna in modified_structures:
            P = find_valid_point(dna, left_indices, right_indices, find_longest_strand(dna)[0])
            new_dna_structures = remove_one_strand_in_sphere(dna, P, sphere_radius)
            new_structures.extend(new_dna_structures)
        if not new_structures:
            break
        output_paths = export_dna_structures(new_structures, base_path)
        sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations(new_structures, base_path, sim_base_path)
        for dna in new_structures:
            point_pos, bend_angle = find_bend_angle(dna, left_indices, right_indices, find_longest_strand(dna)[0])
            results.append((point_pos, bend_angle))
            if abs(bend_angle - target_angle) <= tolerance:
                print(f"Achieved target bend angle: {bend_angle} at point {point_pos}")
                return results
        modified_structures = new_structures
    return results

In [ ]:
# Run the main workflow
results = main(base_path, sim_base_path)

# Plot the results
steps = [res[0] for res in results]
angles = [res[2] for res in results]

# Enhanced Plotting
plt.figure(figsize=(10, 6))
plt.plot(steps, angles, marker='o', linestyle='-', color='b', label='Bend Angle')
plt.axhline(y=target_angle, color='r', linestyle='--', label='Target Angle')
plt.xlabel('Modification Step')
plt.ylabel('Bend Angle (degrees)')
plt.title('Bend Angle vs. Modification Step')
plt.legend()
plt.grid(True)

# Annotate points where the bend angle is within the desired tolerance
for i, angle in enumerate(angles):
    if abs(angle - target_angle) <= tolerance:
        plt.annotate(f'{angle:.2f}', (steps[i], angle), textcoords="offset points", xytext=(0,10), ha='center')

plt.show()

In [ ]:
sim_base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/Notebook/Simulations_results/simulations_structure/Automatically_rmvd_staples/one_staple_removed'
base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/one_staple_remvd'

run_all_simulations(new_dna_structures, base_path, sim_base_path)

In [ ]:
sim_base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/Notebook/Simulations_results/simulations_structure/Automatically_rmvd_staples/two_staples_removed'
base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/two_staples_remvd'

run_all_simulations(new_dna_structures_two, base_path, sim_base_path)

In [ ]:
sim_base_path = '/home/ava/MetaBackbone_project/Metabackbone-scripts/Notebook/Simulations_results/simulations_structure/Automatically_rmvd_staples/three_staples_removed'
base_path = '/home/ava/Dropbox (ASU)/temp/Metabackbone/structure_files/six_helix_oxdna_file/Automatically_removed_staples/1512_bp/three_staples_remvd'

run_all_simulations(new_dna_structures_three, base_path, sim_base_path)

In [ ]:
structures = [f'structure_{i}' for i in range(10)]  # Example list of structures
sim_list_rel, sim_list_eq, sim_list_prod = run_all_simulations(structures, base_path, sim_base_path)

In [ ]:
# Plot energy for all relaxation simulations
for sim in sim_list_rel:
    sim.analysis.plot_energy()

In [ ]:
# Plot energy for all equilibration simulations
for sim in sim_list_eq:
    sim.analysis.plot_energy()

In [ ]:
# Plot energy for all prod simulations
for sim in sim_list_prod:
    sim.analysis.plot_energy()